# Convolutional Neural Networks

## Load Data

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, accuracy_score

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


2024-07-23 21:53:36.036624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 21:53:36.165796: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 21:53:36.166324: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 21:53:36.309229: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-23 21:53:37.268124: W tensorflow/compiler/tf

Num GPUs Available:  1


2024-07-23 21:53:38.786139: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-23 21:53:38.946638: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-23 21:53:38.953420: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
images_train = np.expand_dims(np.load("../data/images/images_train.npy") / 255.0, -1)
images_test = np.expand_dims(np.load("../data/images/images_test.npy") / 255.0, -1)
num_samples_train = images_train.shape[0]
num_samples_test = images_test.shape[0]

hog_train = np.load("../data/hog/hog_train.npy").reshape(num_samples_train, -1) * 255 / 255.0
hog_test = np.load("../data/hog/hog_test.npy").reshape(num_samples_test, -1) * 255 / 255.0

y_train = np.load("../data/images/label_ids_train.npy")
y_test = np.load("../data/images/label_ids_test.npy")


In [3]:
labels_train = np.load("../data/images/labels_train.npy")
class_lookup = list(set(zip(y_train, labels_train)))
class_lookup = {id_value: class_value for id_value, class_value in class_lookup}
class_lookup


{7: 'oblique',
 2: 'greenstick',
 4: 'dislocation',
 1: 'spiral',
 0: 'hairline',
 3: 'comminuted',
 5: 'pathological',
 9: 'avulsion',
 8: 'impacted',
 6: 'longitudinal'}

In [4]:
image_input = layers.Input(shape=(images_train.shape[1], images_train.shape[2], images_train.shape[3]))
x = layers.Conv2D(16, (3, 3), activation='relu')(image_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)

hog_input = layers.Input(shape=(hog_train.shape[1],))
x = layers.Concatenate()([x, hog_input])

x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=[image_input, hog_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


2024-07-23 21:53:41.845502: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-23 21:53:41.854928: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-23 21:53:41.861250: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 510, 510,  │        160 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 255, 255,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 253, 253,  │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 126, 126,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 124, 124,  │      9,248 │ max_pooling2d_1[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 492032)    │          0 │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 262144)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 754176)    │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │ 48,267,328 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 10)        │        650 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 48,282,026 (184.18 MB)

 Trainable params: 48,282,026 (184.18 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
model.fit([images_train, hog_train], y_train, validation_split=0.1, epochs=10, batch_size=32)


2024-07-23 21:53:43.681205: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 933232640 exceeds 10% of free system memory.
2024-07-23 21:53:44.816191: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 933232640 exceeds 10% of free system memory.
2024-07-23 21:53:45.666869: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 933232640 exceeds 10% of free system memory.
2024-07-23 21:53:46.098152: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 933232640 exceeds 10% of free system memory.


Epoch 1/10


I0000 00:00:1721786027.613588  507696 service.cc:145] XLA service 0x744020003ba0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721786027.613927  507696 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-23 21:53:47.691335: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-23 21:53:47.839092: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902


 5/28 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.0642 - loss: 6.2717

I0000 00:00:1721786035.018374  507696 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


28/28 ━━━━━━━━━━━━━━━━━━━━ 16s 283ms/step - accuracy: 0.1406 - loss: 4.3812 - val_accuracy: 0.0000e+00 - val_loss: 3.2147
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.7016 - loss: 0.9998 - val_accuracy: 0.0000e+00 - val_loss: 6.2702
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9381 - loss: 0.3117 - val_accuracy: 0.0000e+00 - val_loss: 6.8585
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.9906 - loss: 0.0818 - val_accuracy: 0.0101 - val_loss: 7.3866
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9977 - loss: 0.0356 - val_accuracy: 0.0000e+00 - val_loss: 8.9563
Epoch 6/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.9992 - loss: 0.0209 - val_accuracy: 0.0101 - val_loss: 8.3683
Epoch 7/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9997 - loss: 0.0133 - val_accuracy: 0.0101 - val_loss: 9.1388
Epoch 8/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.9985 - loss: 0.0094 - val_accuracy: 0.0

In [6]:
y_pred = model.predict([images_test, hog_test])
y_pred = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report).transpose()
report.index = report.index.map(lambda x: class_lookup[int(x)] if x.isdigit() else x)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
report


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step
Accuracy: 0.2571428571428571
Classification Report:


/home/davidsolow/.pyenv/versions/3.11.1/envs/fracture-classification/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/davidsolow/.pyenv/versions/3.11.1/envs/fracture-classification/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/davidsolow/.pyenv/versions/3.11.1/envs/fracture-classification/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use 

,precision,recall,f1-score,support
hairline,0.125000,0.100000,0.111111,10.000000
spiral,0.333333,0.166667,0.222222,12.000000
greenstick,0.368421,0.437500,0.400000,16.000000
comminuted,0.235294,0.285714,0.258065,14.000000
dislocation,0.263158,0.526316,0.350877,19.000000
pathological,0.200000,0.277778,0.232558,18.000000
longitudinal,0.250000,0.250000,0.250000,12.000000
oblique,0.363636,0.250000,0.296296,16.000000
impacted,0.000000,0.000000,0.000000,9.000000
avulsion,0.000000,0.000000,0.000000,14.000000
